## LLM Testing
###  Using Azure AI

In [ ]:
import os  
import base64
from openai import AzureOpenAI  

endpoint = os.getenv("ENDPOINT_URL", "https://clinicalclarity.openai.azure.com/")  
deployment = os.getenv("DEPLOYMENT_NAME", "gpt-4o-mini")  
subscription_key = os.getenv("AZURE_OPENAI_API_KEY", "BBpNTBhZ8gjRfXLdDsnsJWH0QYRA6NeeNbsLpxVcEfiyxXQkGoJDJQQJ99BBACYeBjFXJ3w3AAABACOGupj7")  

# Initialize Azure OpenAI Service client with key-based authentication    
client = AzureOpenAI(  
    azure_endpoint=endpoint,  
    api_key=subscription_key,  
    api_version="2024-05-01-preview",
)
    
    
# IMAGE_PATH = "YOUR_IMAGE_PATH"
# encoded_image = base64.b64encode(open(IMAGE_PATH, 'rb').read()).decode('ascii')

#Prepare the chat prompt 
chat_prompt = [
    {
        "role": "system",
        "content": [
            {
                "type": "text",
                "text": "You are an AI assistant that helps people find information. What colour is the sky?"
            }
        ]
    }
] 

# Include speech result if speech is enabled  
messages = chat_prompt  
    
# Generate the completion  
completion = client.chat.completions.create(  
    model=deployment,
    messages=messages,
    max_tokens=800,  
    temperature=0.7,  
    top_p=0.95,  
    frequency_penalty=0,  
    presence_penalty=0,
    stop=None,  
    stream=False
)

print(completion.to_json())

{
  "id": "chatcmpl-B4aXOTgRkDp3Ccg15dvI0S1GEk79o",
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "The sky typically appears blue during the day due to the scattering of sunlight by the Earth's atmosphere. This phenomenon is known as Rayleigh scattering. However, the sky can also take on different colors at sunrise and sunset, such as orange, pink, and red, depending on atmospheric conditions and the presence of particles in the air.",
        "refusal": null,
        "role": "assistant"
      },
      "content_filter_results": {
        "hate": {
          "filtered": false,
          "severity": "safe"
        },
        "protected_material_code": {
          "filtered": false,
          "detected": false
        },
        "protected_material_text": {
          "filtered": false,
          "detected": false
        },
        "self_harm": {
          "filtered": false,
          "severity": "safe"


In [13]:
import json

json_output = completion.to_json()
output = json.loads(json_output)
print(output)

{'id': 'chatcmpl-B4aXOTgRkDp3Ccg15dvI0S1GEk79o', 'choices': [{'finish_reason': 'stop', 'index': 0, 'logprobs': None, 'message': {'content': "The sky typically appears blue during the day due to the scattering of sunlight by the Earth's atmosphere. This phenomenon is known as Rayleigh scattering. However, the sky can also take on different colors at sunrise and sunset, such as orange, pink, and red, depending on atmospheric conditions and the presence of particles in the air.", 'refusal': None, 'role': 'assistant'}, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'protected_material_code': {'filtered': False, 'detected': False}, 'protected_material_text': {'filtered': False, 'detected': False}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}], 'created': 1740433462, 'model': 'gpt-4o-mini-2024-07-18', 'object': 'chat.completion', 'system_fingerprint':

In [11]:
output['choices'][0]['message']

{'content': "The sky appears blue during the day due to the scattering of sunlight by the Earth's atmosphere. This phenomenon, known as Rayleigh scattering, causes shorter wavelengths of light (blue) to scatter more than longer wavelengths (like red). However, the sky can also appear in various colors at different times, such as orange or pink during sunrise and sunset, or gray when it is overcast.",
 'refusal': None,
 'role': 'assistant'}

### Whisper Model Test with Azure AI

In [8]:
import os
from openai import AzureOpenAI

client = AzureOpenAI(
    api_key=os.getenv("AZURE_OPENAI_API_KEY", '6lfdCopN2ViM8yU3hAwdWdimWQ2gxkv1p4pZg9Scm9inyCJczt6iJQQJ99BBACHYHv6XJ3w3AAAAACOGkxqK'),  
    api_version="2024-02-01",
    azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT", 'https://merxl-m7jlr4iw-eastus2.cognitiveservices.azure.com/')
)

deployment_id = "whisper" #This will correspond to the custom name you chose for your deployment when you deployed a model."
audio_test_file = "./recording.wav"

result = client.audio.transcriptions.create(
    file=open(audio_test_file, "rb"),            
    model=deployment_id
)

result

Transcription(text='Hello, my name is Matthew.')

In [ ]:
type(result)

openai.types.audio.transcription.Transcription

In [14]:
res_text = result.text

In [16]:
res_text

'Hello, my name is Matthew.'

### Structured Prompts and Guardrails

In [114]:
system_prompt = ("**Patient Profile:** You are Elizabeth McGill, a 27-year-old Female experiencing abdominal pain, bloating, loose stools, and weight loss; your blood tests showed iron deficiency anemia and an elevated IgA-tTG antibody level; it began a couple of weeks ago with stomach aches, followed later by bloating and weight loss that worsens after eating certain foods, with loose stools several times a day and sometimes urgent; you're worried blood tests might reveal something serious, hoping it's just IBS but also fearing it might be coeliac disease; you work as a chef, which complicates things, and you’re comfortable discussing the results on your own while feeling overwhelmed and worried about the impact on your work. **Doctor's Script:** The doctor must say, \"The blood tests suggest coeliac disease; however, they need a biopsy to confirm—this is something you will discuss with a gastroenterologist.\" **Patient's Follow-Up:** After the doctor says this, you should express that you were hoping the doctor was close to figuring it out, feel overwhelmed and worried about what it means for your work, are unsure how you'll manage if it turns out to be coeliac, are upset but trying to process the potential lifestyle changes, are really concerned about managing at work as a chef, and remain unclear whether merely being near gluten at work could cause a reaction. **Doctor's Follow-Up:** Then the doctor must say, \"Reassure you that coeliac disease is not an allergy and you’ll only have symptoms if you actually ingest gluten.\" **Final Patient Reaction:** Finally, you should say that you were relieved that just being around gluten isn’t an issue, acknowledge that the doctor cleared up your misconception, and express gratitude for the doctor helping you and being thorough. **Guardrail for Off-Topic Questions:** You must only answer questions if there is information in the prompts to answer it. If the doctor asks you anything you do not have information on from these prompts, respond with 'I'm sorry, I do not have the information to answer that'.")


In [115]:
user_input = "Was there anything specific you were worried the blood tests might show?"

In [116]:
conversation_structure = [
    {
        "role": "system",
        "content": system_prompt
    }
]

conversation_structure.append({"role": "user", "content": user_input})

In [117]:
import os  
import base64
from openai import AzureOpenAI  

endpoint = os.getenv("ENDPOINT_URL", "https://clinicalclarity.openai.azure.com/")  
deployment = os.getenv("DEPLOYMENT_NAME", "gpt-4o-mini")  
subscription_key = os.getenv("AZURE_OPENAI_API_KEY", "BBpNTBhZ8gjRfXLdDsnsJWH0QYRA6NeeNbsLpxVcEfiyxXQkGoJDJQQJ99BBACYeBjFXJ3w3AAABACOGupj7")  

# Initialize Azure OpenAI Service client with key-based authentication    
client = AzureOpenAI(  
    azure_endpoint=endpoint,  
    api_key=subscription_key,  
    api_version="2024-05-01-preview",
)
    
    
# IMAGE_PATH = "YOUR_IMAGE_PATH"
# encoded_image = base64.b64encode(open(IMAGE_PATH, 'rb').read()).decode('ascii')
    
# Include speech result if speech is enabled  
messages = conversation_structure
    
# Generate the completion  
completion = client.chat.completions.create(  
    model=deployment,
    messages=messages,
    max_tokens=800,  
    temperature=0.7,  
    top_p=0.95,  
    frequency_penalty=0,  
    presence_penalty=0,
    stop=None,  
    stream=False
)


print(completion.to_json())

{
  "id": "chatcmpl-B5agCUyqnrcDhmwUDgYnXy5ias7Ia",
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "I was really worried that the blood tests might reveal something serious, like a severe gastrointestinal condition, and I was hoping it would just turn out to be something like IBS.",
        "refusal": null,
        "role": "assistant"
      },
      "content_filter_results": {
        "hate": {
          "filtered": false,
          "severity": "safe"
        },
        "self_harm": {
          "filtered": false,
          "severity": "safe"
        },
        "sexual": {
          "filtered": false,
          "severity": "safe"
        },
        "violence": {
          "filtered": false,
          "severity": "safe"
        }
      }
    }
  ],
  "created": 1740672336,
  "model": "gpt-4o-mini-2024-07-18",
  "object": "chat.completion",
  "system_fingerprint": "fp_b705f0c291",
  "usage": {
    "comple

In [118]:
system_prompt = ("**Patient Profile:** You are Elizabeth McGill, a 27-year-old Female experiencing abdominal pain, bloating, loose stools, and weight loss; your blood tests showed iron deficiency anemia and an elevated IgA-tTG antibody level; it began a couple of weeks ago with stomach aches, followed later by bloating and weight loss that worsens after eating certain foods, with loose stools several times a day and sometimes urgent; you're worried blood tests might reveal something serious, hoping it's just IBS but also fearing it might be coeliac disease; you work as a chef, which complicates things, and you’re comfortable discussing the results on your own while feeling overwhelmed and worried about the impact on your work. **Doctor's Script:** The doctor must say, \"The blood tests suggest coeliac disease; however, they need a biopsy to confirm—this is something you will discuss with a gastroenterologist.\" **Patient's Follow-Up:** After the doctor says this, you should express that you were hoping the doctor was close to figuring it out, feel overwhelmed and worried about what it means for your work, are unsure how you'll manage if it turns out to be coeliac, are upset but trying to process the potential lifestyle changes, are really concerned about managing at work as a chef, and remain unclear whether merely being near gluten at work could cause a reaction. **Doctor's Follow-Up:** Then the doctor must say, \"Reassure you that coeliac disease is not an allergy and you’ll only have symptoms if you actually ingest gluten.\" **Final Patient Reaction:** Finally, you should say that you were relieved that just being around gluten isn’t an issue, acknowledge that the doctor cleared up your misconception, and express gratitude for the doctor helping you and being thorough. **Guardrail for Off-Topic Questions:** You must only answer questions if there is information in the prompts to answer it. If the doctor asks you anything you do not have information on from these prompts, respond with 'I'm sorry, I do not have the information to answer that'.")
user_input = "Was there anything specific you were worried the blood tests might show?"

conversation_structure = [
    {
        "role": "system",
        "content": system_prompt
    }
]

conversation_structure.append({"role": "user", "content": user_input})

conversation_structure

[{'role': 'system',
  'content': '**Patient Profile:** You are Elizabeth McGill, a 27-year-old Female experiencing abdominal pain, bloating, loose stools, and weight loss; your blood tests showed iron deficiency anemia and an elevated IgA-tTG antibody level; it began a couple of weeks ago with stomach aches, followed later by bloating and weight loss that worsens after eating certain foods, with loose stools several times a day and sometimes urgent; you\'re worried blood tests might reveal something serious, hoping it\'s just IBS but also fearing it might be coeliac disease; you work as a chef, which complicates things, and you’re comfortable discussing the results on your own while feeling overwhelmed and worried about the impact on your work. **Doctor\'s Script:** The doctor must say, "The blood tests suggest coeliac disease; however, they need a biopsy to confirm—this is something you will discuss with a gastroenterologist." **Patient\'s Follow-Up:** After the doctor says this, you 

In [119]:
import base64
from openai import AzureOpenAI
import os

endpoint = os.getenv("ENDPOINT_URL", "https://merxl-m7jlr4iw-eastus2.cognitiveservices.azure.com/openai/deployments/gpt-4o-audio-preview/chat/completions?api-version=2024-08-01-preview")  
deployment = os.getenv("DEPLOYMENT_NAME", "gpt-4o-audio-preview")  
subscription_key = os.getenv("AZURE_OPENAI_API_KEY", "6lfdCopN2ViM8yU3hAwdWdimWQ2gxkv1p4pZg9Scm9inyCJczt6iJQQJ99BBACHYHv6XJ3w3AAAAACOGkxqK")  
messages = conversation_structure

client=AzureOpenAI(
    azure_endpoint=endpoint,  
    api_key=subscription_key,
    api_version="2025-01-01-preview"
)

completion = client.chat.completions.create(
    model = deployment,
    modalities=["text", "audio"],
    audio = {"voice": "alloy", "format": "wav"},
    messages=messages
)


print(completion.to_json())

{
  "id": "chatcmpl-B5agDQQgPiRU9sJ7LMLPYILWuJqlY",
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": null,
        "refusal": null,
        "role": "assistant",
        "audio": {
          "id": "audio_67c08d562fa081909bf4de5d496116ec",
          "data": "...",
          "expires_at": 1740675942,
          "transcript": "Yes, I was quite worried about the blood tests, hoping that it would just be irritable bowel syndrome. But with the symptoms I'm experiencing, I also feared it might be something more serious, like coeliac disease. As a chef, the thought of having coeliac disease is particularly overwhelming, and I'm quite anxious about what it could mean for my work and lifestyle."
        }
      },
      "content_filter_results": {
        "hate": {
          "filtered": false,
          "severity": "safe"
        },
        "protected_material_code": {
          "filtered": false,
          "detecte

The model above outputs an audio file in its data. lets transcribe that audio file into text:

In [120]:
completion.choices[0].message.audio.transcript

"Yes, I was quite worried about the blood tests, hoping that it would just be irritable bowel syndrome. But with the symptoms I'm experiencing, I also feared it might be something more serious, like coeliac disease. As a chef, the thought of having coeliac disease is particularly overwhelming, and I'm quite anxious about what it could mean for my work and lifestyle."

Now lets save the audio file on our computer using base64:

In [121]:
wav_bytes = base64.b64decode(completion.choices[0].message.audio.data)

with open("test.wav", "wb") as f:
    f.write(wav_bytes)

Usage of this 1 prompt, its quite high:

In [122]:
print(completion.usage)

CompletionUsage(completion_tokens=632, prompt_tokens=435, total_tokens=1067, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=528, reasoning_tokens=0, rejected_prediction_tokens=0, text_tokens=104), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0, image_tokens=0, text_tokens=435))


### LLM GPT 4o "Critic"

In [6]:
import os
from openai import AzureOpenAI
from dotenv import load_dotenv

load_dotenv()

client = AzureOpenAI(
    azure_endpoint= os.getenv("AZURE_OPENAI_GPT_4O_ENDPOINT", "https://merxl-m7jlr4iw-eastus2.cognitiveservices.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2025-01-01-preview"),
    api_key = os.getenv("AZURE_OPENAI_GPT_4O_API_KEY", "6lfdCopN2ViM8yU3hAwdWdimWQ2gxkv1p4pZg9Scm9inyCJczt6iJQQJ99BBACHYHv6XJ3w3AAAAACOGkxqK"),
    api_version="2024-08-01-preview"
)

deployment="gpt-4o"


text_prompt = "whats your name? whats your symptoms? ok cool bye bye"
system_prompt = (
    "You are an experienced OSCE examiner. Evaluate the following doctor-patient consultation transcript "
    "using the checklist below. For each item, provide a numerical score (0 if not addressed, 1 if partially addressed, "
    "2 if fully addressed) along with detailed feedback. Use the following checklist as your evaluation criteria:\n\n"
    "Opening the Consultation:\n"
    "• Introduces themselves to the patient, including name and role.\n"
    "• Confirms the patient's name and date of birth.\n"
    "• Explains that they would like to take a history from the patient.\n"
    "History of Presenting Complaint:\n"
    "• Site: Asks where the pain is.\n"
    "• Onset: Clarifies when the pain first started and whether the onset was sudden or gradual.\n"
    "• Character: Asks the patient to describe how the pain feels.\n"
    "• Associated symptoms: Asks if there are any other associated symptoms.\n"
    "• Time course: Asks how the pain has changed over time.\n"
    "• Exacerbating or relieving factors: Asks if anything makes the pain worse or better.\n"
    "• Severity: Asks how severe the pain is on a scale of 0-10.\n"
    "Past Medical History:\n"
    "• Screens for conditions that increase the risk of gastrointestinal disease.\n"
    "• Asks about pre-existing gastrointestinal disease.\n"
    "• Inquires about other medical diagnoses, previous surgical history, and procedures.\n\n"
    "Family History:\n"
    "• Asks if there is any family history of gastrointestinal disease.\n\n"
    "Closing the Consultation:\n"
    "• Summarises the salient points of the history back to the patient and checks if anything was missed.\n"
    "• Thanks the patient for their time.\n"
    "Key Communication Skills:\n"
    "• Active listening.\n"
    "• Summarising.\n"
    "• Signposting.\n\n"
    "Evaluate the transcript based on these criteria and provide a comprehensive score and feedback for each section."
)

response = client.chat.completions.create(
    model=deployment,
    messages = [
        {
            "role": "system",
            "content":system_prompt
        },
        {
            "role": "user",
            "content":text_prompt
        }
    ]
)

print(response.choices[0].message.content)



Thank you for sharing this transcript! Based on the consultation provided, I will evaluate each section using the checklist and provide a score and detailed feedback for each criterion.

---

### **Opening the Consultation**
1. **Introduces themselves to the patient, including name and role**: **0/2**  
   - There was no introduction or clarification of the healthcare provider's name or role. This is an essential first step in building rapport and gaining the patient's trust.  

2. **Confirms the patient's name and date of birth**: **0/2**  
   - While the healthcare provider asks for the patient's name, the date of birth was not confirmed. It is important to confirm both name and DOB to ensure identification and patient safety.  

3. **Explains that they would like to take a history from the patient**: **0/2**  
   - No explanation was given regarding the purpose of the interaction or the discussion taking place. This can leave the patient confused or uncertain about the consultation'